# Longitudinal Clonal Dynamics Assessment by Clonotype Tumor Presence Status

### Set Up Env

In [ ]:
# Load project configuration
setwd("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE_TCR")
options(repr.matrix.max.rows=100, repr.matrix.max.cols=100)
options(warn = -1)
source("code/helper/Config.R", echo = FALSE)

In [ ]:
# Load extra libraries
library(vegan)
library(ggpubr) 
library(rstatix)

In [ ]:
# Import plotting helper functions
source("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE_TCR/code/helper/Plotting_Functions.R", echo = FALSE)

In [ ]:
# Define figures path
fig_dir <- "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE_TCR/out/figs/TCR_Fig_Jan"

### Load Tumor 10x Processed TCR Data with matched GEX Profiles 

In [ ]:
# Read data (wide)
data <- qread(file = file.path(root_dir, "out", "data", "SERP_TCR-GEX_wide_11-2025_v2.qs"))

In [ ]:
# Read data (long)
data_long <- qread(file = file.path(root_dir, "out", "data", "SERP_TCR-GEX_11-2025_v2.qs"))

### Prepare data

In [ ]:
# Remove outlier
data %>%
    slice_max(cloneSize_T0, n = 5) %>%
    select(clonotype_id, patient, cloneSize_T0) %>%
    head()
data %>%
    slice_max(cloneSize_T1, n = 5) %>%
    select(clonotype_id, patient, cloneSize_T1) %>%
    head()
data <- data %>%
    filter(clonotype_id != "TRBV2_TRBJ1-1_GCCAGGGGGGATGATCCGAAGGGGCGGGGGACTGAAGCTTTC")
dim(data)

In [ ]:
# Substitute NAs from normalized counts to 0s
data <- data %>%
    mutate(across(c(norm_cloneSize_T0, norm_cloneSize_T1, norm_cloneSize_EOT),
                ~ replace_na(., 0)))

In [ ]:
# Create consensus lv1 annotation between SCR and C02 for pre-existing clonotypes
data <- data %>%
    mutate(lv1 = ifelse((presence_status == "Pre-existing") & (lv1_T0 == lv1_T1), lv1_T1, ifelse(
        presence_status == "De Novo", lv1_T1, ifelse(
            presence_status == "Lost", lv1_T0, "not_matched")
    )))

In [ ]:
# Create consensus CD4/CD8 annotation between SCR and C02 for pre-existing clonotypes
data <- data %>%
    mutate(CD4_CD8_assignment = ifelse((presence_status == "Pre-existing") & (CD4_CD8_assignment_T0 == CD4_CD8_assignment_T1), CD4_CD8_assignment_T1, ifelse(
        presence_status == "De Novo", CD4_CD8_assignment_T1, ifelse(
            presence_status == "Lost", CD4_CD8_assignment_T0, "not_matched")
    )))

In [ ]:
# Filter data for CD4 and CD8 populations, SCR and C02 time points, liver and lung mets, and patients with matched SCR and C02 data
patients_keep <- c("P01", "P02", "P03", "P10", "P14", "P17", "P20", "P26", "P29", "P31", "P33", "P34", "P35") 
data_filt <- data %>%
    filter(
        patient %in% patients_keep,
        met_loc %in% c("Lung", "Liver") 
    )
dim(data_filt)

## Quantification of clonal classes of non-persistent, persistent and de novo clonotypes across T lineages and metastatic sites

In [ ]:
data_long <- data_long %>% 
    # Compute dataset-driven clonal classes
    mutate(cloneClass_2 = case_when(
       cloneSize == 1 ~ "Singlet", 
       cloneSize > 1 ~ cut(
           log10(norm_cloneSize),
           breaks = quantile(log10(norm_cloneSize[cloneSize > 1]), 
                             probs = c(0,0.5,1), na.rm = TRUE),
           labels = c("Lowly Expanded (< Median)", "Highly Expanded (> Median)")     
           )
        )
    )
table(data_long$cloneClass_2)

In [ ]:
exp_data <- data_long %>% 
    filter(
        patient %in% patients_keep,
        met_loc %in% c("Lung", "Liver") 
    ) %>%
    filter(timepoint != "EOT", CD4_CD8_assignment %in% c("CD4", "CD8")) %>%
    mutate(patient, expansion_status = ifelse(presence_status %in% c("De Novo", "Lost"), presence_status, ifelse(timepoint == "T0", "Pre-existing_SCR", "Pre-existing_C02"))) %>%# Create expansion_status column
    group_by(patient, met_loc, cloneClass_2, expansion_status, CD4_CD8_assignment) %>%
    summarise(n = n(), .groups = "drop") %>%
    group_by(met_loc, expansion_status, CD4_CD8_assignment) %>%
    mutate(prop = n / sum(n)) %>%
    group_by(met_loc, cloneClass_2, expansion_status, CD4_CD8_assignment) %>%
    summarise(median_prop = median(prop)) %>%
    filter(cloneClass_2 != "NA") %>%
    as.data.frame()
    
head(exp_data,15)
dim(exp_data)

In [ ]:
options(repr.plot.width = 7, repr.plot.height = 5)

# Set factor levels for consistent ordering
exp_data$cloneClass <- factor(exp_data$cloneClass,
                                     levels = rev(levels(factor(exp_data$cloneClass))))
exp_data$expansion_status <- factor(exp_data$expansion_status,
                                       levels = c("Lost", "Pre-existing_SCR", "Pre-existing_C02", "De Novo"))

# Plot
ggplot(exp_data, aes(x = expansion_status, y = median_prop, fill = cloneClass_2)) + 
    geom_bar(stat = "identity", position = "fill", linewidth = 0.25, color = "black", size = 5) + 
    theme_linedraw(base_size = 15) + 
    theme(
        legend.position = "right",
        panel.grid = element_blank(),
        panel.border = element_rect(color = "black", linewidth = 1.5),
        #axis.line = element_line(color = "black", linewidth = 1.5),
        axis.text.x = element_text(angle = 30, hjust = 1, vjust = 1),
        #axis.text.y = element_text(size = 8),
        strip.background = element_rect(fill = "lightgray", color = "black"),
        strip.text = element_text(color = "black", face = "bold"),
        #legend.title = element_text(face = "bold"),
        #legend.text = element_text(size = 10),
        axis.title.x = element_blank(),
        #axis.title.y = element_text(size = 10),
    ) +
    scale_y_continuous(expand = expansion(mult = c(0.02, 0.05))) +
    scale_fill_manual(
        values = c("Singlet" = "#140b34", "Lowly Expanded (< Median)" = "#84206b", "Highly Expanded (> Median)" = "#f19f4e"),
        labels = c("Singlet" = "Singlet", "Lowly Expanded (< Median)" = "Lowly Expanded", "Highly Expanded (> Median)" = "Highly Expanded")
    ) +
    scale_x_discrete(labels = c(
        "Lost" = "Non-persistent",
        "Pre-existing_SCR" = "Persistent (SCR)",
        "Pre-existing_C02" = "Persistent (C02)",
        "De Novo" = "De Novo"
    )) +
    ylab("Scaled Median Patient Proportions") +
    guides(fill = guide_legend(title = "Clonal Class")) + 
    facet_grid(met_loc ~ CD4_CD8_assignment)

# Save the plot
ggsave(
    filename = file.path(fig_dir, "Barplot_Expansion_Status_by_MetLoc-T-Lineage_2.pdf"),
    plot = last_plot(),
    dpi = 300,
    width = 7, height = 5
)

## Compute proportions of each presence status group

In [ ]:
pct <- data_filt %>%
    group_by(patient, presence_status, met_loc) %>%
    summarise(n = n(), .groups = "drop") %>%
    group_by(patient) %>%
    mutate(pct = n / sum(n)) %>%
    group_by(patient, met_loc) %>%
    mutate(clr_pct = log(pct) - mean(log(pct))) %>%
    ungroup()
head(pct)

In [ ]:
# Plot % of presence statuses
options(repr.plot.width = 5, repr.plot.height = 4)
pct$presence_status <- factor(pct$presence_status, levels = c("Lost", "Pre-existing", "De Novo"))
ggplot(pct, aes(x = presence_status, y = pct,)) +
        geom_boxplot(width = 3, outlier.shape = NA, color = "black", alpha = 0.3) +
        geom_dotplot(aes(fill = met_loc), binaxis='y', stackdir='center', position=position_dodge(0)) + 
        scale_fill_manual(
            values = c("Lung" = "#1a80bb", "Liver" = "#a00000")        
        ) +
        theme_linedraw(base_size = 15) + 
        theme(
            legend.position = "none",
            panel.border = element_rect(color = "black", fill = NA, size = 1),
            plot.title = element_text(hjust=0.5)
        ) + 
        ylim(0,1.1) + 
        labs(x = "Presence Status", y = "Proportion") + 
        stat_compare_means(
            method = "wilcox.test", 
            paired = TRUE, 
            label = "p.format",
            comparisons = list(c("Lost", "Pre-existing"), c("Pre-existing", "De Novo"), c("Lost", "De Novo")),
            label.y = c(0.78, 0.92, 1)
        )
        
ggsave(filename = file.path(fig_dir, "2_Pct_Presence_Status_Tissue.pdf"), plot = last_plot(), dpi = 300, width = 5, height = 4)

## Clonality assessment between non-persistent and de novo clones and persistent clones across SCR and C02 time points

In [ ]:
# Compute clonality per presence status group
clonality_t <- data_filt %>%
    filter(CD4_CD8_assignment %in% c("CD4", "CD8")) %>%
    group_by(patient, presence_status, met_loc, CD4_CD8_assignment) %>%
    summarise(
        richness_T0 = specnumber(norm_cloneSize_T0),
        richness_T1 = specnumber(norm_cloneSize_T1),
        shannon_T0 = diversity(norm_cloneSize_T0, index = "shannon", base=exp(1)),
        shannon_T1 = diversity(norm_cloneSize_T1, index = "shannon", base=exp(1)),
        clonality_T0 = 1 - (shannon_T0 / log(richness_T0)),
        clonality_T1 = 1 - (shannon_T1 / log(richness_T1)),

        
    ) %>%
    pivot_longer(
        cols = c(clonality_T0, clonality_T1),
        names_to = c(".value", "timepoint"),
        names_pattern = "(.*)_(T\\d)"
    ) %>%
    select(-c(richness_T0, richness_T1, shannon_T0, shannon_T1)) %>%
    mutate(
        presence_status_tp = paste(presence_status, timepoint, sep = "_"),
        met_t = paste(met_loc, CD4_CD8_assignment)
    ) %>%
    filter(clonality != 1)
head(clonality_t)

In [ ]:
# Plot clonality of presence statuses
options(repr.plot.width = 6, repr.plot.height = 4)
clonality_t$presence_status_tp <- factor(clonality_t$presence_status_tp, levels = c("Lost_T0", "Pre-existing_T0", "Pre-existing_T1", "De Novo_T1"))
ggplot(clonality_t %>% filter(presence_status != "Pre-existing"), aes(x = met_loc, y = clonality, fill = presence_status_tp)) +
        geom_boxplot(width = 2, outlier.shape = NA, color = "black", alpha = 0.3, size = 1) +
        geom_dotplot(binaxis='y', stackdir='center', position=position_dodge(1)) + 
        scale_fill_manual(
            values = c("Lost_T0" = "#a559aa", "Pre-existing" = "#f0c571", "De Novo_T1" = "#59a89c"),
            labels = c("Lost_T0" = "Non-persistent", "De Novo_T1" = "De Novo")
        ) +
        theme_linedraw(base_size = 15) + 
        theme(
            legend.position = "bottom",
            panel.border = element_rect(color = "black", fill = NA, size = 1.5),
            plot.title = element_text(hjust=0.5),
            axis.text.x = element_text(angle = 0, hjust = 0.5, vjust = 1),
            strip.background = element_rect(fill = "lightgray", color = "black"),
            strip.text = element_text(color = "black", face = "bold")
        ) +
        labs(x = "", y = "Clonality", fill = "") + 
        facet_wrap(~CD4_CD8_assignment)
ggsave(filename = file.path(fig_dir, "Clonality_Lost_vs_DeNovo_Tissue-T-Lineage.pdf"), plot = last_plot(), dpi = 300, width = 6, height = 4)

In [ ]:
# Compute statistical tests
p_val_lung <- wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
p_val_lung
p_val_liver <- wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
p_val_liver
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
                     clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)

In [ ]:
clonality_t_wide <- clonality_t %>%
    select(-timepoint, -presence_status_tp) %>%
    filter(presence_status %in% c("De Novo", "Lost"), CD4_CD8_assignment == "CD8") %>%
    pivot_wider(
        names_from = presence_status, 
        values_from = clonality,
        names_prefix = "Clonality_"
    ) %>%
    # create clonality delta
    mutate(clonality_delta = `Clonality_De Novo`, Clonality_Lost)
head(clonality_t_wide)

In [ ]:
library(boot)

# Function to compute median Δ and 95% CI per tissue, using precomputed p-value
compute_delta_stats <- function(df, tissue, p_val, delta_col = "clonality_delta") {
  
  # Subset delta values
  delta_vals <- df %>%
    filter(met_loc == tissue) %>%
    pull({{delta_col}})
  
  # Median
  med <- median(delta_vals)
  
  # Bootstrap CI
  median_fun <- function(data, indices) median(data[indices])
  set.seed(123)
  boot_obj <- boot(delta_vals, statistic = median_fun, R = 10000)
  ci <- boot.ci(boot_obj, type = "perc")$percent[4:5]  # 95% CI lower/upper
  
  # Return as tibble
  tibble(
    Tissue = tissue,
    Median_Delta = med,
    CI_lower = ci[1],
    CI_upper = ci[2],
    Wilcox_p = p_val
  )
}

# Compute stats for liver and lung using your precomputed p-values
stats_liver <- compute_delta_stats(clonality_t_wide, "Liver", p_val_liver, "clonality_delta")
stats_lung  <- compute_delta_stats(clonality_t_wide, "Lung",  p_val_lung, "clonality_delta")

# Combine into a single table
delta_stats_table <- bind_rows(stats_liver, stats_lung)

# Print table
print(delta_stats_table)

In [ ]:
# Plot clonality of presence statuses
options(repr.plot.width = 6, repr.plot.height = 4)
clonality_t$presence_status_tp <- factor(clonality_t$presence_status_tp, levels = c("Lost_T0", "Pre-existing_T0", "Pre-existing_T1", "De Novo_T1"))
clonality_t_1 <- clonality_t %>% filter(presence_status_tp %in% c("Lost_T0", "De Novo_T1"))
clonality_t$presence_status_tp <- factor(clonality_t$presence_status_tp, levels = c("Lost_T0", "Pre-existing_T0", "Pre-existing_T1", "De Novo_T1"))
ggplot(clonality_t_1, aes(x = met_loc, y = clonality, fill = presence_status_tp)) +
        geom_boxplot(width = 2, outlier.shape = NA, color = "black", alpha = 0.3) +
        geom_dotplot(binaxis='y', stackdir='center', position=position_dodge(1)) + 
        scale_fill_manual(
            values = c("Lost_T0" = "#1a80bb", "De Novo_T1" = "#ea801c"),
            labels = c("Lost_T0" = "Lost", "De Novo_T1" = "De Novo")
        ) +
        theme_linedraw(base_size = 15) + 
        theme(
            legend.position = "bottom",
            panel.border = element_rect(color = "black", fill = NA, size = 1),
            plot.title = element_text(hjust=0.5)
        ) + 
        labs(x = "Presence Status", y = "Clonality", fill = "Status") + 
        facet_wrap(~CD4_CD8_assignment)
ggsave(filename = file.path(fig_dir, "Clonality_Lost_vs_DeNovo_Tissue-T-Lineage.pdf"), plot = last_plot(), dpi = 300, width = 6, height = 4)

In [ ]:
# Comput effect sizes
clonality_cd4_pers_scr_liver <- clonality_t %>% filter(CD4_CD8_assignment == "CD4", met_loc == "Liver", presence_status_tp == "Pre-existing_T0") %>% pull(clonality)
clonality_cd4_pers_c02_lung <- clonality_t %>% filter(CD4_CD8_assignment == "CD4", met_loc == "Lung", presence_status_tp == "Pre-existing_T1") %>% pull(clonality)


delta <- median(clonality_cd4_pers_scr_liver) - median(clonality_cd4_pers_c02_lung)
cat("Delta:", delta)


# Compute non-parametric CI
set.seed(123)
n_boot <- 10000

boot_delta <- replicate(n_boot, {
  lung_star  <- sample(clonality_cd4_pers_c02_lung,  replace = TRUE)
  liver_star <- sample(clonality_cd4_pers_scr_liver, replace = TRUE)
  median(liver_star, na.rm = TRUE) - median(lung_star, na.rm = TRUE)
})

# 95% percentile CI
ci <- quantile(boot_delta, probs = c(0.025, 0.975))

cat("\n95% CI:", ci, "\n")

In [ ]:
# Plot clonality of presence statuses
options(repr.plot.width = 6, repr.plot.height = 4)
clonality_t$presence_status_tp <- factor(clonality_t$presence_status_tp, levels = c("Lost_T0", "Pre-existing_T0", "Pre-existing_T1", "De Novo_T1"))
ggplot(clonality_t %>% filter(presence_status == "Pre-existing"), aes(x = met_loc, y = clonality, fill = presence_status_tp)) +
        geom_boxplot(width = 2, outlier.shape = NA, color = "black", alpha = 0.3, size = 1) +
        geom_dotplot(binaxis='y', stackdir='center', position=position_dodge(1)) + 
        scale_fill_manual(
            values = c("Lost_T0" = "#a559aa", "Pre-existing_T1" = "#1a80bb", "Pre-existing_T0" = "#ea801c", "De Novo_T1" = "#59a89c"),
            labels = c("Pre-existing_T0" = "Persistent (SCR)", "Pre-existing_T1" = "Persistent (C02)")
        ) +
        theme_linedraw(base_size = 15) + 
        theme(
            legend.position = "bottom",
            panel.border = element_rect(color = "black", fill = NA, size = 1.5),
            plot.title = element_text(hjust=0.5),
            axis.text.x = element_text(angle = 0, hjust = 0.5, vjust = 1),
            strip.background = element_rect(fill = "lightgray", color = "black"),
            strip.text = element_text(color = "black", face = "bold")
        ) +
        labs(x = "", y = "Clonality", fill = "") + 
        facet_wrap(~CD4_CD8_assignment) + 
        ylim(0, 0.7)
ggsave(filename = file.path(fig_dir, "Clonality_Pre-Exh_T0vsT1_Tissue-T-Lineage.pdf"), plot = last_plot(), dpi = 300, width = 6, height = 4)

In [ ]:
# Tests for Lost vs De Novo and Prexisting SCR vs C02 comparisons

# Pre-existing CD4
print("Pre-Existing CD4")
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)

# Pre-existing CD8
print("Pre-Existing CD8")
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Pre-existing_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)

# Pre-existing CD8
print("Lost vs De Novo CD8")
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD8", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD8", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)

# Pre-existing CD8
print("Lost vs De Novo CD4")
wilcox.test(clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = TRUE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)
wilcox.test(clonality_t %>% filter(met_loc == "Liver", CD4_CD8_assignment == "CD4", presence_status_tp == "De Novo_T1") %>% pull(clonality), 
            clonality_t %>% filter(met_loc == "Lung", CD4_CD8_assignment == "CD4", presence_status_tp == "Lost_T0") %>% pull(clonality), 
            paired = FALSE)$p.value %>% signif(3)